# ======================================================
# Project: Critical Transitions – ICU Outcomes
# Author: Alanoud Alturki
# ======================================================



## -----------------------------
## Project Objectives
## -----------------------------
### Primary Objectives:
### 1. Identify demographic, clinical, laboratory, treatment, and comorbidity
###    factors associated with prolonged ICU stay (>7 days) among critically ill adults.
### 2. Determine predictors of 30-day post-discharge outcomes (readmission or mortality)
###    among ICU survivors, compared to patients who remain alive without readmission.

### Secondary / Exploratory Objectives:
### 3. Assess whether polypharmacy, immunosuppression, and comorbidity burden
###    modify the risks of prolonged ICU stay and adverse 30-day outcomes.
### 4. Evaluate predictive model performance (ROC curves, calibration,
###    decision curve analysis) and develop clinically interpretable
###    visual tools (nomograms, survival curves, probability plots).

## Overall Goal:
### To generate evidence-based insights into ICU patient trajectories,
### quantify risk factors for prolonged stay and adverse post-discharge outcomes,
### and identify modifiable predictors to inform early intervention
### and improve continuity of critical care.

## -----------------------------
## Note on Outcome Definition
## -----------------------------
### The outcome variable is a three-level factor:
### - Alive: survived without readmission in 30 days
### - Readmitted: discharged alive but readmitted within 30 days
### - Dead: died within 30 days (in-hospital or post-discharge)

In [4]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# -----------------------------
# Step 1. Connect to PostgreSQL
# -----------------------------
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="mimiciv",
)

# -----------------------------
# Step 2. Demographics + Admissions + ICU
# -----------------------------
query_base = """
SELECT
    ie.subject_id,
    ie.hadm_id,
    ie.stay_id,
    ROUND(CAST(ie.los AS numeric),2) AS icu_los,
    pat.gender,
    pat.anchor_age AS age,
    adm.admission_type,
    adm.insurance,
    adm.admittime,
    adm.dischtime,
    adm.deathtime,
    ie.first_careunit,
    ie.last_careunit
FROM mimiciv_icu.icustays ie
JOIN mimiciv_hosp.admissions adm
    ON ie.hadm_id = adm.hadm_id
JOIN mimiciv_hosp.patients pat
    ON ie.subject_id = pat.subject_id
WHERE pat.anchor_age >= 18;
"""
df = pd.read_sql(query_base, conn)

# -----------------------------
# Step 3. Labs (first 24h averages)
# -----------------------------
query_labs = """
SELECT
    hadm_id,
    AVG(CASE WHEN itemid = 50811 THEN valuenum END) AS wbc,
    AVG(CASE WHEN itemid = 50882 THEN valuenum END) AS hemoglobin,
    AVG(CASE WHEN itemid = 51265 THEN valuenum END) AS platelets,
    AVG(CASE WHEN itemid = 50912 THEN valuenum END) AS creatinine,
    AVG(CASE WHEN itemid = 51006 THEN valuenum END) AS bun,
    AVG(CASE WHEN itemid = 50931 THEN valuenum END) AS sodium,
    AVG(CASE WHEN itemid = 50971 THEN valuenum END) AS potassium,
    AVG(CASE WHEN itemid = 50813 THEN valuenum END) AS lactate,
    AVG(CASE WHEN itemid = 50878 THEN valuenum END) AS ast,
    AVG(CASE WHEN itemid = 50861 THEN valuenum END) AS alt,
    AVG(CASE WHEN itemid = 50863 THEN valuenum END) AS alp,
    AVG(CASE WHEN itemid = 50885 THEN valuenum END) AS bilirubin,
    AVG(CASE WHEN itemid = 50862 THEN valuenum END) AS albumin
FROM mimiciv_hosp.labevents
WHERE valuenum IS NOT NULL
GROUP BY hadm_id;
"""
labs = pd.read_sql(query_labs, conn)

# -----------------------------
# Step 4. Vitals (first 24h averages)
# -----------------------------
query_vitals = """
SELECT
    stay_id,
    AVG(CASE WHEN itemid IN (220045,211) THEN valuenum END) AS heart_rate,
    AVG(CASE WHEN itemid IN (220210,615) THEN valuenum END) AS resp_rate,
    AVG(CASE WHEN itemid IN (220052,456) THEN valuenum END) AS map,
    AVG(CASE WHEN itemid IN (223761,678) THEN valuenum END) AS temperature,
    AVG(CASE WHEN itemid IN (220277,646) THEN valuenum END) AS spo2,
    AVG(CASE WHEN itemid IN (223900,454) THEN valuenum END) AS gcs
FROM mimiciv_icu.chartevents
WHERE valuenum IS NOT NULL
GROUP BY stay_id;
"""
vitals = pd.read_sql(query_vitals, conn)

# -----------------------------
# Step 5. Treatments
# -----------------------------
query_treat = """
SELECT
    ce.stay_id,
    MAX(CASE WHEN itemid IN (30047,30051,30055) THEN 1 ELSE 0 END) AS mech_vent,
    MAX(CASE WHEN itemid IN (225113, 225114, 225118) THEN 1 ELSE 0 END) AS dialysis,
    MAX(CASE WHEN itemid IN (30044,30042,30119) THEN 1 ELSE 0 END) AS vasopressor
FROM mimiciv_icu.procedureevents ce
GROUP BY ce.stay_id;
"""
treatments = pd.read_sql(query_treat, conn)

# -----------------------------
# Step 6. Comorbidities
# -----------------------------
query_comorb = """
SELECT
    di.hadm_id,
    MAX(CASE WHEN icd_code LIKE 'I1%' OR icd_code LIKE '401%' THEN 1 ELSE 0 END) AS hypertension,
    MAX(CASE WHEN icd_code LIKE 'E1%' OR icd_code LIKE '250%' THEN 1 ELSE 0 END) AS diabetes,
    MAX(CASE WHEN icd_code LIKE 'I50%' OR icd_code LIKE '428%' THEN 1 ELSE 0 END) AS chf,
    MAX(CASE WHEN icd_code LIKE 'N18%' OR icd_code LIKE '585%' THEN 1 ELSE 0 END) AS ckd,
    MAX(CASE WHEN icd_code LIKE 'J44%' OR icd_code LIKE '496%' THEN 1 ELSE 0 END) AS copd,
    MAX(CASE WHEN icd_code LIKE 'C%' THEN 1 ELSE 0 END) AS cancer,
    MAX(CASE WHEN icd_code LIKE 'K7%' OR icd_code LIKE '571%' THEN 1 ELSE 0 END) AS liver_disease,
    MAX(CASE WHEN icd_code LIKE 'D8%' OR icd_code LIKE '279%' THEN 1 ELSE 0 END) AS immunosuppression
FROM mimiciv_hosp.diagnoses_icd di
GROUP BY di.hadm_id;
"""
comorb = pd.read_sql(query_comorb, conn)

# -----------------------------
# Step 7. Medications (Polypharmacy)
# -----------------------------
query_meds = """
SELECT
    pr.hadm_id,
    COUNT(DISTINCT pr.drug) AS n_medications
FROM mimiciv_hosp.prescriptions pr
WHERE pr.drug IS NOT NULL
GROUP BY pr.hadm_id;
"""
meds = pd.read_sql(query_meds, conn)

# -----------------------------
# Step 8. Readmission
# -----------------------------
query_readmit = """
SELECT
    subject_id,
    hadm_id,
    admittime,
    dischtime,
    LEAD(admittime) OVER (PARTITION BY subject_id ORDER BY admittime) AS next_admit
FROM mimiciv_hosp.admissions;
"""
readmit = pd.read_sql(query_readmit, conn)
readmit["readmit_30d"] = (
    (pd.to_datetime(readmit["next_admit"]) - pd.to_datetime(readmit["dischtime"])).dt.days <= 30
).astype(int)
readmit = readmit[["hadm_id","readmit_30d"]]

# -----------------------------
# Step 9. Merge All Sources
# -----------------------------
df = df.merge(labs, on="hadm_id", how="left")
df = df.merge(vitals, on="stay_id", how="left")
df = df.merge(treatments, on="stay_id", how="left")
df = df.merge(comorb, on="hadm_id", how="left")
df = df.merge(meds, on="hadm_id", how="left")
df = df.merge(readmit, on="hadm_id", how="left")

# Polypharmacy flag
df["polypharmacy"] = (df["n_medications"] >= 5).astype(int)

# -----------------------------
# Step 10. Derived Outcomes
# -----------------------------
df["prolonged_icu_los"] = (df["icu_los"] > 7).astype(int)
df["hosp_los"] = (pd.to_datetime(df["dischtime"]) - pd.to_datetime(df["admittime"])).dt.days
df["prolonged_hosp_los"] = (df["hosp_los"] > 7).astype(int)
df["death_30d"] = (
    df["deathtime"].notna() &
    ((pd.to_datetime(df["deathtime"]) - pd.to_datetime(df["dischtime"])).dt.days <= 30)
).astype(int)

# -----------------------------
# Step 11. Handle Missing Data
# -----------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns
imputer = IterativeImputer(max_iter=10, random_state=42)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# -----------------------------
# Step 12. Deduplicate (most recent ICU stay)
# -----------------------------
df["admittime"] = pd.to_datetime(df["admittime"])
df = df.sort_values(["subject_id","admittime"], ascending=[True,True])
df = df.groupby("subject_id").tail(1).reset_index(drop=True)

# -----------------------------
# Step 13. Export
# -----------------------------
df.to_csv("/Users/alanoudalturki/Desktop/mimic_full_icu_enhanced.csv", index=False)

print("Final dataset exported with:")
print("- Prolonged ICU LOS")
print("- 30-day mortality")
print("- 30-day readmission")
print("- Polypharmacy")
print("- Immunosuppression")
print("- Comorbidities + labs + vitals + treatments")
print("Final shape:", df.shape)

/var/folders/3_/0cs1sd196_scl_z87dl52tv80000gn/T/ipykernel_5516/388747017.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query_base, conn)
/var/folders/3_/0cs1sd196_scl_z87dl52tv80000gn/T/ipykernel_5516/388747017.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  labs = pd.read_sql(query_labs, conn)
/var/folders/3_/0cs1sd196_scl_z87dl52tv80000gn/T/ipykernel_5516/388747017.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vitals = pd.read_sql(query_vitals, conn)
/var/folders/3_/0cs1sd196_scl_z87dl

Final dataset exported with:
- Prolonged ICU LOS
- 30-day mortality
- 30-day readmission
- Polypharmacy
- Immunosuppression
- Comorbidities + labs + vitals + treatments
Final shape: (65366, 50)


In [5]:
df.head()

,subject_id,hadm_id,stay_id,icu_los,gender,age,admission_type,insurance,admittime,dischtime,...,cancer,liver_disease,immunosuppression,n_medications,readmit_30d,polypharmacy,prolonged_icu_los,hosp_los,prolonged_hosp_los,death_30d
0,10000032.0,29079034.0,39553978.0,0.41,F,52.0,EW EMER.,Medicaid,2180-07-23 12:35:00,2180-07-25 17:55:00,...,0.0,1.0,0.0,17.0,1.0,1.0,0.0,2.0,0.0,0.0
1,10000690.0,25860671.0,37081114.0,3.89,F,86.0,EW EMER.,Medicare,2150-11-02 18:02:00,2150-11-12 13:45:00,...,0.0,0.0,0.0,37.0,0.0,1.0,0.0,9.0,1.0,0.0
2,10000980.0,26913865.0,39765666.0,0.50,F,73.0,EW EMER.,Medicare,2189-06-27 07:38:00,2189-07-03 03:00:00,...,0.0,0.0,0.0,34.0,0.0,1.0,0.0,5.0,0.0,0.0
3,10001217.0,27703517.0,34592300.0,0.95,F,55.0,DIRECT EMER.,Private,2157-12-18 16:58:00,2157-12-24 14:55:00,...,0.0,0.0,0.0,27.0,0.0,1.0,0.0,5.0,0.0,0.0
4,10001725.0,25563031.0,31205490.0,1.34,F,46.0,EW EMER.,Private,2110-04-11 15:08:00,2110-04-14 15:00:00,...,0.0,0.0,0.0,35.0,0.0,1.0,0.0,2.0,0.0,0.0


In [6]:
# Show as Index object
print(df.columns)

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_los', 'gender', 'age',
       'admission_type', 'insurance', 'admittime', 'dischtime', 'deathtime',
       'first_careunit', 'last_careunit', 'wbc', 'hemoglobin', 'platelets',
       'creatinine', 'bun', 'sodium', 'potassium', 'lactate', 'ast', 'alt',
       'alp', 'bilirubin', 'albumin', 'heart_rate', 'resp_rate', 'map',
       'temperature', 'spo2', 'gcs', 'mech_vent', 'dialysis', 'vasopressor',
       'hypertension', 'diabetes', 'chf', 'ckd', 'copd', 'cancer',
       'liver_disease', 'immunosuppression', 'n_medications', 'readmit_30d',
       'polypharmacy', 'prolonged_icu_los', 'hosp_los', 'prolonged_hosp_los',
       'death_30d'],
      dtype='object')


In [7]:
from summarytools import dfSummary

summary = dfSummary(df)
print(summary)

In [8]:
#Summary After 
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,subject_id[float64],Mean (sd) : 14998493.4 (2887822.5)min < med < max:10000032.0 < 15000454.0 < 19999987.0IQR (CV) : 5007297.8 (5.2),"65,366 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACh0lEQVR4nO3YPW8aQRRG4TuA8fCRXYEJ7mhSWkqBREOTLmV+bNp0sWSk1BRpUrmxLCFYxMaIhZDGkaiA3bHBr3Ke2rP3AmeQhdtutwa8daVzLwAco3LuBfB2OOdqZlYNeMRqu90+vdQ+uw6GGrh8ycz+FDwbej50duE3PfA9e7UPex/nXO3q6upLFEWtos+Yz+dT59zX19h/b6ghy282m4s0TT80m81fpVIpO+X50NlmZrPZbOGc+2Zmy5xHfbvd/hzHcePEc/8pekGjZrPZHQ6H8ziOc89OksTf3t52J5NJxzk3LzB/7wU99I1ajaKoNRwOn/Iuf39/3xqPx+/7/f7P6+vraZ6zoedDZz88PLwbjUafer1ep1wu53rdWZZ5M7sZDAY/Op1Org8sZK5Z2AXNssxvNpuby8vL791u93fe2WmaXmRZ9rHX65WL7H7o2/io/1HjOF7mXX46ndbMzBqNRu6zoedfYrb3vt7v95cFL0nde58V2bvo3J3ZIZe7vl6vy3nnmpmtVqtK0d2TJPF3d3etyWRSNbPiof6vQi7Jqefuzj7X3kVnP9s7n5+nIIFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIaFyzB8lSeLzPnixWHgzszRN/ePjY/2U51Vnq+4dev6Yvv4CQTdL+Ep8z6YAAAAASUVORK5CYII="">",0(0.0%)
2,hadm_id[float64],Mean (sd) : 24976595.5 (2886129.3)min < med < max:20000094.0 < 24976100.5 < 29999828.0IQR (CV) : 5003282.0 (8.7),"65,366 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACQ0lEQVR4nO3Yva/SUBjH8eeUKzm82IYXYWNxdGvCwuLm6B/r6g6JM4OLEwshIaWEeokE6+R4ubQV21/y/cycnofk+xDA5XluQNMFdQ8A3OPpkQ93znXMrF3hEb/yPH/+V/NA16uhVojND4fDT1EU9UqcNTOzw+Fwcs59NbNzieOBmf0ue7dVWJKKC1rb3E12M1TnXGc0Gn0Ow3BQ9MGXy8Wb2Yf5fP5tPB4fi57fbrdvV6vVx9lsNm61WoVCvV6vb7Ise9/v938EQXAperdZpSUpvaA1z23W4CV57RO1HYbhYLFYPEdRVOiNbzabwXq97nrvL5PJ5GfRwZIk6Xjvu3Ecn6fTaVLi7ndxHH8vetas2pJUWdA6527AktyM/K7vqFEUnYvGliRJp8jrX9Lr9UrfXebs3/MVl6TUgjZg7lqWxMzseDwmzrkvL8X60B9T6qosSZ3UljtNU79cLgf7/b5tZoSKxysbupndXHD+R4UEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoWEp3telKapL/rg0+nkzcyyLPO73a77P8+r3q06d9Xz9/T1B+dJPDTP5pzMAAAAAElFTkSuQmCC"">",0(0.0%)
3,stay_id[float64],Mean (sd) : 34848801.3 (2886756.5)min < med < max:30000153.0 < 34769458.0 < 39999858.0IQR (CV) : 5004747.2 (12.1),"65,366 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACeklEQVR4nO3aMW/aQBiH8feAIBOoESKFjaV7B0aWfoN+2K4dI8EXYMjSiQUhIWOEGwRy6ZJWqlQBvsOBP3p+a7jDw+PLa8AdDgcDbl3l2hcAnKN26gXOuYaZ1T333x0Oh1fPtcBfR0N1zjW63e7XOI47Ppuv1+vEOfeNWBHq1Ilaj+O4MxqNXtvt9rbIxmmaRuPxuLNcLutmRqgIcvJfv5lZu93e9nq9n0U3z/M8NrPYOVf4wt5UzOyX51rGjjtyVqg+six72O/3nweDQbVarRY6jc3M8jx/yLLsU6vV+lGpVPZF169Wq41z7ruZFX7vN4R+Q0oLdbfb1aIoehwOh9t+v58UXT+bzTrT6fTjcDh8Kbp+Pp9/mEwmXwaDwZPPTWLGfH1rSgv1j2az6TU2JEnS8F2fJEkj5CZJ0zR6fn7uLZfLJ+fcuuh64zS+uNJDvSbfmyR0bAkcO4j8P+46VF8hY0vo2HGB2fouH0AJ9Yj3HjtCIw99AL3luZxQS3KN2TrkAfQCc7lZiScyod6gazyAhs7lZuXO5oQKMwv/ODF0bDk1dhAq/hFymvuGfs7X7YSKi/IN3cwax/7I71EhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhoXbOi9I0jYpuvNlsIjOzLMuixWLx+J7rVd9b9bpD15/T12+fR3k0vLD1xgAAAABJRU5ErkJggg=="">",0(0.0%)
4,icu_los[float64],Mean (sd) : 3.5 (5.1)min < med < max:0.0 < 1.9 < 226.4IQR (CV) : 2.7 (0.7),"2,781 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcn